# Perform MC simulation on the $\mu$ VT ensemble using HOOMD-Blue and user defined potential (hard sphere double ramp Jagla model)

### Import necessary libraries and initialise hoomd on CPU.

In [ ]:
import hoomd
import hoomd.hpmc
import hoomd.jit
import hoomd.hpmc.field
import numpy as np
import ase
import ase.io
from matplotlib import pyplot 

In [ ]:
hoomd.context.initialize('--mode=cpu');

### Inout parameters: temperature, fugacity and initial xyz configuration. ###
Note that the temperature is set by scaling the Jagla parameters appropriately - this is down when initialising the potential.

In [ ]:
at=ase.io.read("Jagla_hoomd_LDliq_N585.extxyz")
Temp=0.42 # temperature
Fugacity=0.000025 # fugacity

In [ ]:
# create HOOMD system
cell = at.get_cell()
pos = at.get_positions()
nd = 1 # if reading in a smaller cell, it can be multiplied to create a larger simulation box.
uc = hoomd.lattice.unitcell(N=len(at),
                            a1=cell[0],
                            a2=cell[1],
                            a3=cell[2],
                            dimensions=3,
                            position=pos,
                            type_name=['A']*len(at));
system = hoomd.init.create_lattice(unitcell=uc, n=[nd, nd, nd]);

In [ ]:
# set HOOMD MC integrator 
# initial stepsize = d
mc = hoomd.hpmc.integrate.sphere(seed=266671, d=0.1)
# hard sphere diameter = diameter
mc.shape_param.set('A', diameter=1.0);

### Jagla potential - set parameters
patch.alpha_iso[0] = depth of potential well and patch.alpha_iso[1] = height of the repulsive ramp. In order to control the temperature of the simulation the calculation corresponds to, both of these parameters are divided by Temp.

r_cut = cutoff distance the potential is truncated

In [ ]:
# T* = kT/E    
Jagla       = """float rsq = dot(r_ij, r_ij);
                     if (sqrt(rsq) >= 1.72f)    
                         return -alpha_iso[0] + alpha_iso[0]*((sqrt(rsq)-1.72f)/(3.0f-1.72f));
                     else
                         return alpha_iso[1] - ( alpha_iso[1] + alpha_iso[0]) * (sqrt(rsq) - 1.0f)/(1.72f-1.0f);
              """

patch = hoomd.jit.patch.user(mc=mc, r_cut=3.0, array_size=2, code=Jagla)
patch.alpha_iso[0]=1.0/Temp
patch.alpha_iso[1]=3.5/Temp
print(patch.alpha_iso)

In [ ]:
# quantities to be logged during MC run
quantities=["hpmc_patch_energy","volume","num_particles","pressure","hpmc_overlap_count"]
log = hoomd.analyze.log(filename="LDliq585_GC.out",quantities=quantities,period=100)

In [ ]:
# set the stepsize for atom moves
mc.set_params(d=0.13)
# define constant chemical potential and set the fugacity to be kept fixed
muvt = hoomd.hpmc.update.muvt(mc, seed=83701,period=1);
muvt.set_fugacity(type='A', fugacity=Fugacity)
# particle insertions/deletions attempted at every period, which is set 4 cycles by default in HOOMD
# Set the period to 1 to try insertion/deletion more frequently
muvt.set_period(period=1)
mc.set_params(nselect=1)

### Run simulation ###

In [ ]:
hoomd.run(1000)

In [ ]:
# current state of the simulation
U = log.query(quantity="hpmc_patch_energy") # potential energy (this is scaled by Temp!)
OC = log.query(quantity="hpmc_overlap_count") # overlap count of hard spheres (should be zero)
V = log.query(quantity="volume") # volume of simulation box 
n_at = len(system.particles)  # current number of particles
print(U, OC, V, n_at)  

In [ ]:
def save_config(hoomd_system,atom_types={"A":"H","B":"He"}):
    
    lattice=np.array([hoomd_system.box.get_lattice_vector(i=i) for i in range(3)])
    
    pos=[system.particles[i].position for i in range(hoomd_system.particles.types.pdata.getN())]
    types=[atom_types[system.particles[i].type] for i in range(hoomd_system.particles.types.pdata.getN())]
    ase_atoms = ase.Atoms(pbc=[(True,True,True)],cell=lattice,positions=pos, symbols=types)
    
    ase_atoms.wrap()
    return ase_atoms

In [ ]:
current_config = save_config(system)
ase.io.write("Jagla_LDliq585_GC.final.extxyz", current_config)